In [1]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# 크롬 옵션
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 ...")

# 쿠키: 네가 준 전체 쿠키 반영
cookies = [
    {"name": "__T_", "value": "1", "domain": ".page.kakao.com", "path": "/", "httpOnly": False, "secure": False},
    {"name": "__T_SECURE", "value": "1", "domain": ".page.kakao.com", "path": "/", "httpOnly": False, "secure": True},
    {"name": "_kahai", "value": "f341a75bb3cd3a4bd1a22bab3aceeb6e20dfb5e2ace653fc4cca086b03fd53b1", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_karmt", "value": "-9pyWzFlNyRBKRcWTSRwmvZc2o3eYdntSFBwoKbh_V2N1PIWMc53TGMuTttJQDCd", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kau", "value": "73106297ba50319faabb0d29da81edf43b0a3662a795a16010d184f71adb834de7f1d57d56c56f5c593a57828709bdc6a482275b137e168115a16dd2bc1ea49be71ed11d6e6f8ea53c61d6d984f35764456253bf49042401a361b814b2bdc8225bf2cf7ec338494b2ad7b0e8f8d4e95b8550eea4e1d2f519e79880b135313731343732363033343632303736373632343139323030343337303735335f92c5aed2e435d9620a7f08bf3e985e", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kawlt", "value": "1eQLPBaT48WGIYhLnBUnjG4sepEwtuYy7yJafx0GqZIVClAcPotAm7SVyEEtFEmPtDILIWdSG76n_ZpbvkuqRYvItKyqekFCVaejFBX_KSwrYlq_tm0eKwrdt82sJWkm", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kpawbat", "value": "usvjafCMhgHJ3fExC6Gjwi-I_z3vqjUiAAAAAQoNIJsAAAGXKODQAujqOP6o1CZo", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kpwtkn", "value": "U2FsdGVkX1%2FNTXj3oJOv%2BZclt4W6IIk9uv7qA%2BnSjvhYG741Yg4OtnrCnM76qR8e7FQ6UX7dfugTX54enNo0kPVQE40fKGQneVBe4n53vty0DCc8ywUEIq35vWbXOUAx%2B8VpCm86QCw98VEz9hmYBu28Cq0Q8C40Orjad02OpSlRhMPz2zmPvSJGawGaXhjeYf%2F8JLoHlspo4Xnk%2FUAjxVUzk8ShLb3UQ6aS0UripYJcGVQaldwmpuC29gI%2B88Ew9MR2FUtEcYben5YHfgoHJGFXoc18HjKFNOiWVlwDCl4I5AM3X5q9fBAqlzMqS99rusqipwnJ0wJyzZevvCsvdyF%2FQZFrPX3WcT%2BsONX%2FODoBZRAJhP5ooqXf%2F4xCgwXRO4%2FGQaS%2FvTESUsn89DA2PTwwtLwPVtGmZZcnjozGZYhlkoN9jcFY7o1%2BhsmyVa5naWz9Jn%2B2TvHLoUK4H5vygh7X3wDnT8ZxH38lqw6iDJJ%2F%2FoNQHcsm5n7pX5zFpAvvqDcgTwAMgP8HUPy0BSPpB2WbPGVBZFDg0FBRfAnYHUUYAzURpxIw0hDv0d%2FQ33g785pOHrb4LgjOyL3wWnWa4AHCtiXiMTChwwS02wq6ewe%2FZ7QOGCzCjdoUt7JHyULKcclP%2FIArrbxg6gmfZUqnQKSCv9GPQrEUejmc912KIcPpoYr2uDBd7Ar0TQHHLUAy9s3LSa7GYDq9voSQqPw6a7MuyuxT5FNlt21skiY%2F9CVVfzGy2HrDR%2BQGyotexBnEJtcYJMz4FeIrNYUohpABNTz2vcN6LEvoMnl1VEz77kcqmD9CegYWwqya2UtSXMMnjmDx0WLstwGAcXWcY2vmw699t%2BOujOFD%2FQyRD0FFtSmVFyabTWTnaeVXHE2uig8e0q35ywoYFhMQwxc9qVtcIppiegjzM1pa4DJ3nLyfoVAwnSaCGL%2FF3BWY1mBxfHNccYh9ijKHV1R5rknVAg559x4rk95r4DLGU2qpWnS5xioj7iwqCLgzBY%2B1XS0Wy0g%2F5zp5IKZxoAZ0mk%2F2xxClzoMgWOUfCDquQEef%2F0Q8amPJhCuWRjVveRTN9QuaDuaT3aYrISXWltJejhQ3fGqBTahhOwSVTXMmn4Pud15HeXlgx6ztV4KWdkM7C2k3VbxSm0iIYciNqb%2BrDzN31S4AWC2UK2IqIiwBRcYclnCmcrF3839zMt9sZ6HnLzx349g%2FGo5ec5ZeFqXTf7Y8CLx9acAlutglombX1iyS8WhA9ZW6ULrI2vv8OXwi6I0BDPmPSGKrbMnhXJZJgunQhiYUhtergHkEgzoM5N0fD7BLXJ4%2BG32aovU%2BOe3DJDWhpOcTJU1BKIHFaGkOweEPesozk5Sds%2BoqIR5NQH%2BGxRITW%2BwkvKzRDz4ihFGeeR3WN2T8K6HKx0mf8h9QZRUHBmp8jjCjCF5HLNcQDy%2FGJoTflXpiYIyiFV5ZviWa4jhxPZ%2FZLW1eRqPwzUn%2B3ykJjP0Yr4UKmF18u0WPHaKknuPfZaBo93c%2FQgZhtIMhZCFu5HuCA0CopCGZTMWF2jIf9EUn17OQle%2Bv6x76qnBO%2BQ8LsOmNl7T3rjnYtZ5TtRZNszNQBKYTJ%2Bf%2BpToRD2IYGM%2BbJu1LOufGqfIenIEctUxywTIILrHqbPGvcWoIe8d2rYtXdtLmKA82K4iE5ZCG4sp0RrxuFvyxI7%2FNMAdIu82zod5JN3SDwf1Q1UnDePRXv0WoZ8xwDBnOS%2FTys6zLFH1yRBx8c1YhNCHeP0vaJgxwahC2L7IBRLncrt0OmL%2F2qVIjGuOu%2BC%2FUt2fhQJ7COoTYtLBqOwN9yw%2FCcVDthT3IoWoCYRR5livnm37U29OkzmHidkoOc739uPHbYoh6TwRrhA%3D%3D", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True}
]

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get("https://page.kakao.com/")

for cookie in cookies:
    driver.add_cookie(cookie)

title_ids = [62477346, 65332544, 64867816, 66119536, 66110723]

records = []

for title_id in title_ids:
    url = f"https://page.kakao.com/content/{title_id}"
    driver.get(url)
    time.sleep(2)

    try:
        # 작품 제목 (현 페이지)
        title_elem = driver.find_element(By.CSS_SELECTOR, "span.font-large3-bold")
        current_title = title_elem.text.strip()

        # "이 작가의 다른 작품" 영역 찾기
        try:
            header_div = driver.find_element(By.XPATH, "//div[text()='이 작가의 다른 작품']")
            ancestor_div = header_div.find_element(By.XPATH, "./ancestor::div[contains(@class, 'w-full overflow-hidden')]")
            parent_of_list = ancestor_div.find_element(By.XPATH, "./following-sibling::div//ul")
            li_list = parent_of_list.find_elements(By.TAG_NAME, "li")
        except Exception:
            li_list = []

        count = 0
        for li in li_list:
            try:
                info_div = li.find_element(By.CSS_SELECTOR, "div[aria-hidden='true'].w-full.space-y-4pxr.pb-4pxr.pr-8pxr.pt-8pxr.h-76pxr")
                media_elem = info_div.find_element(By.CSS_SELECTOR, "span.break-all.align-middle")
                media = media_elem.text.strip()
                if media == "웹소설":
                    continue

                # 부가 요소 ([ ]나 ( )) 제거 후도 카운팅 제외
                title_elem = info_div.find_element(By.CSS_SELECTOR, "div.font-small1.line-clamp-2.break-all.text-el-60")
                raw_title = title_elem.text.strip()
                cleaned_title = re.sub(r'(\[.*?\]|\(.*?\))', '', raw_title).strip()
                if cleaned_title:
                    count += 1
            except Exception:
                continue

        # 현재 페이지 작품 +1
        final_count = count + 1
        records.append({"title_id": title_id, "작가_작품수": final_count})
        print(f"[{title_id}] / 작가_작품수: {final_count}")

    except Exception as e:
        print(f"[{title_id}] Error: {e}")
        records.append({"title_id": title_id, "작가_작품수": 1})  # 에러나도 최소 1개

df = pd.DataFrame(records)
df.to_csv("final_작가작품수.csv", index=False, encoding="utf-8-sig")
driver.quit()


[62477346] / 작가_작품수: 7
[65332544] / 작가_작품수: 2
[64867816] / 작가_작품수: 1
[66119536] / 작가_작품수: 1
[66110723] / 작가_작품수: 11


In [ ]:
# 쿠키: 네가 준 전체 쿠키 반영
cookies = [
    {"name": "__T_", "value": "1", "domain": ".page.kakao.com", "path": "/", "httpOnly": False, "secure": False},
    {"name": "__T_SECURE", "value": "1", "domain": ".page.kakao.com", "path": "/", "httpOnly": False, "secure": True},
    {"name": "_kahai", "value": "f341a75bb3cd3a4bd1a22bab3aceeb6e20dfb5e2ace653fc4cca086b03fd53b1", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_karmt", "value": "-9pyWzFlNyRBKRcWTSRwmvZc2o3eYdntSFBwoKbh_V2N1PIWMc53TGMuTttJQDCd", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kau", "value": "73106297ba50319faabb0d29da81edf43b0a3662a795a16010d184f71adb834de7f1d57d56c56f5c593a57828709bdc6a482275b137e168115a16dd2bc1ea49be71ed11d6e6f8ea53c61d6d984f35764456253bf49042401a361b814b2bdc8225bf2cf7ec338494b2ad7b0e8f8d4e95b8550eea4e1d2f519e79880b135313731343732363033343632303736373632343139323030343337303735335f92c5aed2e435d9620a7f08bf3e985e", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kawlt", "value": "1eQLPBaT48WGIYhLnBUnjG4sepEwtuYy7yJafx0GqZIVClAcPotAm7SVyEEtFEmPtDILIWdSG76n_ZpbvkuqRYvItKyqekFCVaejFBX_KSwrYlq_tm0eKwrdt82sJWkm", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kpawbat", "value": "usvjafCMhgHJ3fExC6Gjwi-I_z3vqjUiAAAAAQoNIJsAAAGXKODQAujqOP6o1CZo", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kpwtkn", "value": "U2FsdGVkX1%2FNTXj3oJOv%2BZclt4W6IIk9uv7qA%2BnSjvhYG741Yg4OtnrCnM76qR8e7FQ6UX7dfugTX54enNo0kPVQE40fKGQneVBe4n53vty0DCc8ywUEIq35vWbXOUAx%2B8VpCm86QCw98VEz9hmYBu28Cq0Q8C40Orjad02OpSlRhMPz2zmPvSJGawGaXhjeYf%2F8JLoHlspo4Xnk%2FUAjxVUzk8ShLb3UQ6aS0UripYJcGVQaldwmpuC29gI%2B88Ew9MR2FUtEcYben5YHfgoHJGFXoc18HjKFNOiWVlwDCl4I5AM3X5q9fBAqlzMqS99rusqipwnJ0wJyzZevvCsvdyF%2FQZFrPX3WcT%2BsONX%2FODoBZRAJhP5ooqXf%2F4xCgwXRO4%2FGQaS%2FvTESUsn89DA2PTwwtLwPVtGmZZcnjozGZYhlkoN9jcFY7o1%2BhsmyVa5naWz9Jn%2B2TvHLoUK4H5vygh7X3wDnT8ZxH38lqw6iDJJ%2F%2FoNQHcsm5n7pX5zFpAvvqDcgTwAMgP8HUPy0BSPpB2WbPGVBZFDg0FBRfAnYHUUYAzURpxIw0hDv0d%2FQ33g785pOHrb4LgjOyL3wWnWa4AHCtiXiMTChwwS02wq6ewe%2FZ7QOGCzCjdoUt7JHyULKcclP%2FIArrbxg6gmfZUqnQKSCv9GPQrEUejmc912KIcPpoYr2uDBd7Ar0TQHHLUAy9s3LSa7GYDq9voSQqPw6a7MuyuxT5FNlt21skiY%2F9CVVfzGy2HrDR%2BQGyotexBnEJtcYJMz4FeIrNYUohpABNTz2vcN6LEvoMnl1VEz77kcqmD9CegYWwqya2UtSXMMnjmDx0WLstwGAcXWcY2vmw699t%2BOujOFD%2FQyRD0FFtSmVFyabTWTnaeVXHE2uig8e0q35ywoYFhMQwxc9qVtcIppiegjzM1pa4DJ3nLyfoVAwnSaCGL%2FF3BWY1mBxfHNccYh9ijKHV1R5rknVAg559x4rk95r4DLGU2qpWnS5xioj7iwqCLgzBY%2B1XS0Wy0g%2F5zp5IKZxoAZ0mk%2F2xxClzoMgWOUfCDquQEef%2F0Q8amPJhCuWRjVveRTN9QuaDuaT3aYrISXWltJejhQ3fGqBTahhOwSVTXMmn4Pud15HeXlgx6ztV4KWdkM7C2k3VbxSm0iIYciNqb%2BrDzN31S4AWC2UK2IqIiwBRcYclnCmcrF3839zMt9sZ6HnLzx349g%2FGo5ec5ZeFqXTf7Y8CLx9acAlutglombX1iyS8WhA9ZW6ULrI2vv8OXwi6I0BDPmPSGKrbMnhXJZJgunQhiYUhtergHkEgzoM5N0fD7BLXJ4%2BG32aovU%2BOe3DJDWhpOcTJU1BKIHFaGkOweEPesozk5Sds%2BoqIR5NQH%2BGxRITW%2BwkvKzRDz4ihFGeeR3WN2T8K6HKx0mf8h9QZRUHBmp8jjCjCF5HLNcQDy%2FGJoTflXpiYIyiFV5ZviWa4jhxPZ%2FZLW1eRqPwzUn%2B3ykJjP0Yr4UKmF18u0WPHaKknuPfZaBo93c%2FQgZhtIMhZCFu5HuCA0CopCGZTMWF2jIf9EUn17OQle%2Bv6x76qnBO%2BQ8LsOmNl7T3rjnYtZ5TtRZNszNQBKYTJ%2Bf%2BpToRD2IYGM%2BbJu1LOufGqfIenIEctUxywTIILrHqbPGvcWoIe8d2rYtXdtLmKA82K4iE5ZCG4sp0RrxuFvyxI7%2FNMAdIu82zod5JN3SDwf1Q1UnDePRXv0WoZ8xwDBnOS%2FTys6zLFH1yRBx8c1YhNCHeP0vaJgxwahC2L7IBRLncrt0OmL%2F2qVIjGuOu%2BC%2FUt2fhQJ7COoTYtLBqOwN9yw%2FCcVDthT3IoWoCYRR5livnm37U29OkzmHidkoOc739uPHbYoh6TwRrhA%3D%3D", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True}
]